<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">DESPLIEGUE DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [7]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Mounted at /content/drive


In [12]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import openpyxl
import logging
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import os

#### CONEXIÓN CON LA BASE DE DATOS

In [19]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db')

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [21]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',), ('ratings_movies',)]

In [22]:
table_movies = pd.read_sql('SELECT * FROM movies', conexion)
table_movies.head()

,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,Toy Story,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,Jumanji,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,Grumpier Old Men,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,Waiting to Exhale,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,Father of the Bride Part II,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
table_ratings = pd.read_sql('SELECT * FROM ratings', conexion)
table_ratings.head()

,userId,movieId,rating,day,month,hour,year
0,1,1,4.0,30,7,18,2000
1,1,3,4.0,30,7,18,2000
2,1,6,4.0,30,7,18,2000
3,1,47,5.0,30,7,19,2000
4,1,50,5.0,30,7,18,2000


In [24]:


# Configuración de logging
logging.basicConfig(
    filename='/content/drive/MyDrive/recomendaciones/script_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def preprocesar(conn=None):
    # Cargar datos desde la base de datos
    movies = pd.read_sql('SELECT * FROM movies', conn)
    ratings = pd.read_sql('SELECT * FROM ratings', conn)

    # Preprocesar películas
    movies['año'] = movies['año'].astype(int)  # Convertir año a tipo entero

    # Escalar el año para tenerlo en un rango común
    sc = MinMaxScaler()
    movies[['año_scaled']] = sc.fit_transform(movies[['año']])

    # Convertir las columnas de géneros a variables dummy
    movies_dum = pd.get_dummies(movies.drop(columns=['movieId', 'titulo', 'año']), drop_first=True)

    log_mes = 'Preprocesamiento de datos completado.'
    logging.info(log_mes)
    print(log_mes)

    return movies_dum, movies, ratings



In [25]:
def recomendar_por_popularidad(user_id, ratings, movies):
    # Obtener las películas más populares (por ejemplo, las más calificadas)
    popular_movies = ratings.groupby('movieId').size().reset_index(name='view_count')
    popular_movies = popular_movies.sort_values(by='view_count', ascending=False).head(10)

    # Unir las películas más populares con la información de las películas
    recomend_b = movies[movies['movieId'].isin(popular_movies['movieId'])]
    recomend_b = recomend_b[['titulo', 'movieId']]  # Mostrar solo el título y el movieId

    log_mes = f'Recomendaciones populares para el usuario {user_id}.'
    logging.info(log_mes)
    print(log_mes)

    return recomend_b


In [26]:
def main(list_user, conexion):
    recomendaciones_todos = pd.DataFrame()

    # Preprocesar los datos
    movies_dum, movies, ratings = preprocesar(conexion)

    # Generar recomendaciones para cada usuario
    for user_id in list_user:
        recomendaciones_popular = recomendar_por_popularidad(user_id, ratings, movies)
        recomendacion_contenido = recomendar_por_contenido(user_id, movies_dum, ratings)

        # Combinar las recomendaciones de popularidad y contenido
        recomendaciones_completas = pd.merge(recomendaciones_popular, recomendacion_contenido, how='outer', on='movieId', suffixes=('_popular', '_contenido'))
        recomendaciones_completas["userId"] = user_id
        recomendaciones_completas.reset_index(inplace=True, drop=True)

        recomendaciones_todos = pd.concat([recomendaciones_todos, recomendaciones_completas])

    # Guardar las recomendaciones en un archivo Excel y CSV
    recomendaciones_todos.to_excel('/content/drive/MyDrive/recomendaciones/recomendaciones.xlsx')
    recomendaciones_todos.to_csv('/content/drive/MyDrive/recomendaciones/recomendaciones.csv')

    log_mes = 'Recomendaciones generadas y guardadas en Excel y CSV.'
    logging.info(log_mes)
    print(log_mes)


In [28]:
if __name__ == "__main__":
    # Usuarios para los que se generarán recomendaciones
    list_user = [1, 2, 3]  # Puedes cambiar estos IDs de usuario según necesites
    conexion = sql.connect('/content/drive/MyDrive/recomendaciones/db_movies.db')  # Asegúrate de ajustar la ruta correctamente
    main(list_user, conexion)


OperationalError: unable to open database file